# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - GraphQL APIs

**Subtopic** - Introduction to GraphQL APIs with Python

**Use Case** - Develop a simple GraphQL API using Python that will read and write data to a local JSON file. The API should be able to perform basic CRUD operations - Create, Read, Update, and Delete. The data structure can be a list of dictionaries, where each dictionary represents a user with fields: 'id', 'name', and 'email'. Use the 'graphene' library for creating the GraphQL schema and resolvers.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt

```


# Prompt

Problem Description:
Implement a Flask‑based GraphQL API in Python that manages user records stored in a local JSON file. The API must support create, read, update and delete operations via GraphQL queries and mutations. Requests must be JSON‑only and validated accordingly. The system must enforce email uniqueness, atomic file writes, in‑memory caching with TTL, a Levenshtein‑based fuzzy search on name and email, bulk email updates, middleware logging, and expose a health‑check endpoint. A background file‑watcher thread must invalidate the cache on external JSON modifications.

- Input Format and Constraints:
All GraphQL operations are sent as POST to /graphql with Content‑Type: application/json. The JSON body must follow GraphQL request structure. No other content types are allowed. The USERS\_JSON\_PATH environment variable may override the default “users.json” file location. The cache TTL is fixed at 5 seconds. No external network calls are permitted.



- Expected Output Format:
Standard GraphQL JSON responses for queries and mutations. The /health endpoint returns JSON with “status” and “timestamp” fields. Errors (invalid JSON, duplicate email, missing user) must return appropriate GraphQL error messages or ValueError for non‑JSON requests.

- Examples:
Query all users with pagination and search:

```graphql
query {
  allUsers(limit: 10, offset: 0, search: "alice") {
    id name email createdAt updatedAt
  }
}
```

Query single user by ID:

```graphql
query {
  userById(id: "uuid‑string") {
    name email
  }
}
```

Create user mutation:

```graphql
mutation {
  createUser(userData: {name: "Bob", email: "bob@example.com"}) {
    ok message user { id name email }
  }
}
```

Update user mutation:

```graphql
mutation {
  updateUser(userData: {id: "uuid", name: "Bob Jr.", email: "bobjr@example.com"}) {
    ok message
  }
}
```

Delete user mutation:

```graphql
mutation {
  deleteUser(id: "uuid") { ok message }
}
```

Bulk email update:

```graphql
mutation {
  bulkUpdateEmails(updates: [
    {id: "uuid1", email: "a@x.com"},
    {id: "uuid2", email: "b@y.com"}
  ]) { ok updatedCount failed }
}
```

# Requirements



Requirements:
Explicit and Implicit Points:

* JSON‑only GraphQL endpoint with Content‑Type check
* CRUD via Graphene schema
* Email uniqueness validation
* Atomic write to JSON file
* In‑memory cache with 5s TTL
* Levenshtein fuzzy search controlled by maxDistance
* Bulk email updates
* Middleware logging of field names and args
* Health‑check route
* File‑watcher thread invalidates cache on file change

Solution Expectations:
The API must behave exactly as defined above. Atomic writes must never corrupt users.json. Cache must serve reads within TTL. Fuzzy and exact searches must filter correctly. Duplicate emails must be rejected. Bulk updates must report successes and failures.

Function Signatures:

```py
def _atomic_write(path: str, data: str) -> None
def load_users(force_reload: bool = False) -> List[Dict[str, Any]]
def save_users(users: List[Dict[str, Any]]) -> None
def find_user_index(user_list: List[Dict[str, Any]], user_id: str) -> int
def levenshtein(a: str, b: str) -> int
```

Class Definitions:

```py
class JSONOnlyGraphQLView(GraphQLView):
    def parse_body(self, request)

class UserType(graphene.ObjectType)
class UserInput(graphene.InputObjectType)
class Query(graphene.ObjectType)
class CreateUser(graphene.Mutation)
class UpdateUser(graphene.Mutation)
class DeleteUser(graphene.Mutation)
class BulkUpdateEmails(graphene.Mutation)
```

Edge Case Behavior:

* Corrupted JSON file resets to empty list
* Non‑JSON requests raise ValueError(“Only 'application/json' supported.”)
* Missing users return null in userById
* Duplicate email mutations return ok=False with message
* Fuzzy search with no matches returns empty list

Constraints:

* Do not use any libraries beyond the specified dependencies and standard library
* No use of reversed(), eval() or exec()
* File watcher must run as daemon thread
* Cache TTL fixed at 5 seconds; not configurable via API

Important Notes:
Assume input JSON is well‑formed GraphQL; only content‑type and business validations are required. On constraint violations, raise ValueError or return GraphQL errors as specified. Tests should not rely on timing longer than TTL + 1s.


In [ ]:
# hybrid_factorizer.py
#!/usr/bin/env python3
"""
Hybrid multi-algorithm integer factorization with blind‑spot thread recovery.

Stages:
  0) trial division
  1) Pollard’s p‑1 with fixed per‑bound timeout
  2) Pollard’s Rho (Brent variant) with heartbeat monitoring and respawn
  3) ECM fallback (stub)

Threads share:
  - stop_event for early cancel
  - result dict under result_lock
  - heartbeat dict under hb_lock
"""

import threading
import time
import math
import random
from typing import Dict, Tuple, Optional, List


def gcd(a: int, b: int) -> int:
    """Compute greatest common divisor of a and b."""
    return math.gcd(a, b)


def trial_division(N: int, limit: int) -> Optional[int]:
    """
    Trial‑divide N by integers 2..limit.
    Return a factor or None if none found ≤ limit.
    """
    r = int(math.isqrt(N))
    for p in range(2, min(limit, r) + 1):
        if N % p == 0:
            return p
    return None


def pollard_p1(
    N: int,
    B: int,
    stop_event: threading.Event,
    heartbeat: Dict[str, float],
    hb_lock: threading.Lock
) -> Optional[int]:
    """
    Pollard's p‑1: compute a = 2^{lcm(1..B)} mod N,
    check g = gcd(a−1, N). Return factor or None.
    """
    a = 2
    for j in range(2, B + 1):
        if stop_event.is_set():
            return None
        a = pow(a, j, N)
        with hb_lock:
            heartbeat[threading.current_thread().name] = time.time()
        g = gcd(a - 1, N)
        if 1 < g < N:
            return g
    return None


def pollard_rho_brent(
    N: int,
    c: int,
    stop_event: threading.Event,
    heartbeat: Dict[str, float],
    hb_lock: threading.Lock
) -> Optional[int]:
    """
    Pollard's Rho with Brent's cycle detection, deterministic start at x=2.
    f(x) = x^2 + c mod N.
    """
    x = 2
    y = x
    m = 100
    g = 1
    r = 1
    q = 1
    f = lambda v: (v * v + c) % N

    while g == 1 and not stop_event.is_set():
        x = y
        for _ in range(r):
            y = f(y)
        k = 0
        while k < r and g == 1 and not stop_event.is_set():
            for _ in range(min(m, r - k)):
                y = f(y)
                q = (q * abs(x - y)) % N
            g = gcd(q, N)
            k += m
            with hb_lock:
                heartbeat[threading.current_thread().name] = time.time()
        r <<= 1

    if g is None or g == 1 or g == N:
        return None
    return g


class PollardP1Worker(threading.Thread):
    """Thread running Pollard’s p‑1 up to bound B."""

    def __init__(
        self,
        N: int,
        B: int,
        stop_event: threading.Event,
        result: Dict[str, int],
        result_lock: threading.Lock,
        heartbeat: Dict[str, float],
        hb_lock: threading.Lock
    ) -> None:
        super().__init__(daemon=True)
        self.N = N
        self.B = B
        self.stop_event = stop_event
        self.result = result
        self.result_lock = result_lock
        self.heartbeat = heartbeat
        self.hb_lock = hb_lock

    def run(self) -> None:
        factor = pollard_p1(
            self.N, self.B,
            self.stop_event,
            self.heartbeat, self.hb_lock
        )
        if factor:
            with self.result_lock:
                if "p" not in self.result:
                    self.result["p"] = factor
                    self.result["q"] = self.N // factor
                    self.stop_event.set()


class PollardRhoWorker(threading.Thread):
    """Thread running Pollard’s Rho (Brent) with constant c."""

    def __init__(
        self,
        N: int,
        c: int,
        stop_event: threading.Event,
        result: Dict[str, int],
        result_lock: threading.Lock,
        heartbeat: Dict[str, float],
        hb_lock: threading.Lock
    ) -> None:
        super().__init__(daemon=True)
        self.N = N
        self.c = c
        self.stop_event = stop_event
        self.result = result
        self.result_lock = result_lock
        self.heartbeat = heartbeat
        self.hb_lock = hb_lock

    def run(self) -> None:
        factor = pollard_rho_brent(
            self.N, self.c,
            self.stop_event,
            self.heartbeat, self.hb_lock
        )
        if factor:
            with self.result_lock:
                if "p" not in self.result:
                    self.result["p"] = factor
                    self.result["q"] = self.N // factor
                    self.stop_event.set()


class HybridFactorizer:
    """
    Hybrid factorization pipeline:
      0) trial division
      1) Pollard p‑1 (fixed timeout per bound)
      2) Pollard Rho with blind‑spot monitoring
      3) (ECM stub)
    """

    def __init__(
        self,
        N: int,
        trial_limit: int = 1000,
        p1_bounds: Tuple[int, int] = (1000, 100_000),
        p1_timeout: float = 1.0,
        rho_workers: int = 4,
        rho_blind_timeout: float = 2.0,
        rho_timeout: float = 10.0,
        monitor_interval: float = 1.0
    ) -> None:
        self.N = N
        self.trial_limit = trial_limit
        self.p1_bounds = p1_bounds
        self.p1_timeout = p1_timeout
        self.rho_workers = rho_workers
        self.rho_blind_timeout = rho_blind_timeout
        self.rho_timeout = rho_timeout
        self.monitor_interval = monitor_interval

        self.stop_event = threading.Event()
        self.result_lock = threading.Lock()
        self.result: Dict[str, int] = {}

        self.heartbeat: Dict[str, float] = {}
        self.hb_lock = threading.Lock()

    def factor(self) -> Tuple[int, int]:
        """Return (p, q) with p*q == N, or (1, N) if no factor found."""

        # Stage 0: trial division
        if self.N <= 1:
            return (self.N, 1)
        if self.N % 2 == 0:
            return (2, self.N // 2)
        f = trial_division(self.N, self.trial_limit)
        if f:
            return (f, self.N // f)

        # Stage 1: Pollard p‑1
        for B in self.p1_bounds:
            if self.stop_event.is_set():
                break
            w = PollardP1Worker(
                self.N, B,
                self.stop_event, self.result, self.result_lock,
                self.heartbeat, self.hb_lock
            )
            with self.hb_lock:
                self.heartbeat[w.name] = time.time()
            w.start()
            w.join(self.p1_timeout)
            if self.stop_event.is_set():
                return (self.result["p"], self.result["q"])
            self.stop_event.set()
            w.join()
            self.stop_event.clear()

        # Stage 2: Pollard Rho with blind‑spot monitoring
        rho_threads: List[PollardRhoWorker] = []
        for _ in range(self.rho_workers):
            c = random.randrange(1, self.N - 1)
            w = PollardRhoWorker(
                self.N, c,
                self.stop_event, self.result, self.result_lock,
                self.heartbeat, self.hb_lock
            )
            with self.hb_lock:
                self.heartbeat[w.name] = time.time()
            rho_threads.append(w)
            w.start()

        start = time.time()
        while time.time() - start < self.rho_timeout and not self.stop_event.is_set():
            time.sleep(self.monitor_interval)
            now = time.time()
            with self.hb_lock:
                for w in list(rho_threads):
                    last = self.heartbeat.get(w.name, 0)
                    if now - last > self.rho_blind_timeout:
                        w.join(0)
                        rho_threads.remove(w)
                        if not self.stop_event.is_set():
                            c = random.randrange(1, self.N - 1)
                            nw = PollardRhoWorker(
                                self.N, c,
                                self.stop_event, self.result, self.result_lock,
                                self.heartbeat, self.hb_lock
                            )
                            self.heartbeat[nw.name] = time.time()
                            rho_threads.append(nw)
                            nw.start()

        self.stop_event.set()
        for w in rho_threads:
            w.join(0.1)

        if "p" in self.result:
            p, q = self.result["p"], self.result["q"]
            return (min(p, q), max(p, q))

        # Stage 3: ECM fallback (not implemented)
        return (1, self.N)


if __name__ == "__main__":
    N = 101 * 103
    f = HybridFactorizer(N)
    p, q = f.factor()
    print(f"Result: {p} × {q}")


ModuleNotFoundError: No module named 'graphene'

In [ ]:
# test_hybrid_factorizer.py
import unittest
import threading
from main import (
    gcd,
    trial_division,
    PollardP1Worker,
    PollardRhoWorker,
    HybridFactorizer
)


class TestHybridFactorizer(unittest.TestCase):

    def test_gcd(self):
        self.assertEqual(gcd(10, 15), 5)
        self.assertEqual(gcd(17, 13), 1)
        self.assertEqual(gcd(0, 5), 5)
        self.assertEqual(gcd(5, 0), 5)

    def test_trial_division(self):
        self.assertEqual(trial_division(15, 5), 3)
        self.assertIsNone(trial_division(17, 5))
        self.assertEqual(trial_division(49, 10), 7)

    def test_factorizer_even_and_one(self):
        f1 = HybridFactorizer(1)
        self.assertEqual(f1.factor(), (1, 1))
        f2 = HybridFactorizer(14)
        self.assertEqual(f2.factor(), (2, 7))

    def test_factorizer_small_composite(self):
        f3 = HybridFactorizer(21, trial_limit=10)
        self.assertEqual(f3.factor(), (3, 7))

    def test_factorizer_medium_composite(self):
        N = 101 * 103
        f4 = HybridFactorizer(N, trial_limit=5, rho_workers=2, rho_timeout=5.0)
        self.assertEqual(f4.factor(), (101, 103))

    def test_factorizer_prime(self):
        f5 = HybridFactorizer(97, trial_limit=10, rho_workers=2, rho_timeout=2.0)
        self.assertEqual(f5.factor(), (1, 97))

    def test_pollard_p1_worker(self):
        N = 13 * 17
        stop = threading.Event()
        hb = {}
        hb_lock = threading.Lock()
        result = {}
        lock = threading.Lock()
        w = PollardP1Worker(N, 50, stop, result, lock, hb, hb_lock)
        w.start()
        w.join(2.0)
        if "p" in result:
            p, q = result["p"], result["q"]
            self.assertIn(p, (13, 17))
            self.assertEqual(p * q, N)

    def test_pollard_rho_worker(self):
        N = 15
        stop = threading.Event()
        hb = {}
        hb_lock = threading.Lock()
        result = {}
        lock = threading.Lock()
        w = PollardRhoWorker(N, 1, stop, result, lock, hb, hb_lock)
        w.start()
        w.join(2.0)
        # now guaranteed to find factor quickly with x=2 start
        self.assertIn("p", result)
        p, q = result["p"], result["q"]
        self.assertIn(p, (3, 5))
        self.assertEqual(p * q, N)


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: <Add the URL here>

#### Model code:

```python
# code generated by the model

# <Issue>: <Add the issue in the model code here>

# code generated by the model
```